## Imports

In [102]:
import os

In [1]:
from sqlalchemy import create_engine
import sqlalchemy
import pandas as pd
import numpy as np
import connectorx as cx
from collections import Counter

In [2]:
from konlpy.tag import Kkma
from konlpy.tag import Mecab
from konlpy.utils import pprint
kkma = Kkma()
mecab = Mecab(dicpath=r"C:\Mecab\mecab-ko-dic")

In [3]:
import pyarrow.parquet as pq

In [4]:
import matplotlib.pyplot as plt
from matplotlib import font_manager

In [5]:
import copy

## Read Files

### Read na_news Parquet

In [8]:
df_na = pq.read_pandas('df_na.parquet').to_pandas()

In [9]:
df_na.head(10)

,id,keyword,title,summary,my_summary,contents,media,date,time,link,...,label_category,label_sentiment,label_etc,direct_predict,time_stamp,job_flag,ai_flag_cate,ai_flag_senti,ai_date_senti,ai_date_cate
0,1,삼성전자,"美, 반도체 리쇼어링 속도… 파운드리 갈 길 바쁜 삼성전자에 기회?",코로나로 반도체 글로벌 공급망 위태… 자국 내 위탁생산시설 설립 추진 박차 1위 T...,반도체 업계에서는 파운드리 사업을 큰 축으로 ‘2030년 비메모리 1위’를 목표로 ...,코로나로 반도체 글로벌 공급망 위태… 자국 내 위탁생산시설 설립 추진 박차\n\n1...,뉴스플러스,2020-05-11,2020-05-11 11:50:51,https://biz.chosun.com/site/data/html_dir/2020...,...,None,None,None,NaN,2020-05-11 21:50:32,Y,Y,Y,2021-05-28,2021-06-02
1,5,삼성전자,"[OA칼럼] 삼성전자, D램 EUV 공정 적용 의미와 전망은",편집자 주 : 전자부품 전문미디어 디일렉은 미국 메모리 반도체 전문 시장조사업체 오...,"삼성전자는 ""처음으로 (1x나노) D램 생산에 극자외선(EUV) 공정을 채택했다""고...",파운드리 사업과 장비 사용 공유\n\n편집자 주 : 전자부품 전문미디어 디일렉은 미...,디일렉,2020-05-12,2020-05-11 20:50:52,http://www.thelec.kr/news/articleView.html?idx...,...,기타,기타,기타,NaN,2020-05-11 21:50:32,Y,Y,Y,2021-05-28,2021-06-02
2,6,삼성전자,"“36개월간 안전하게” KT, 삼성전자 단말보험 '갤럭시케어' 출시",[디지털데일리 최민지기자] KT가 삼성전자 전용 단말보험을 내놓았다. 자기부담금은 ...,"배터리 성능보장 및 수리보증 기간 연장으로, 총 3년간 안심하고 단말을 사용할 수 ...",[디지털데일리 최민지기자] KT가 삼성전자 전용 단말보험을 내놓았다. 자기부담금은 ...,디지털데일리,2020-05-11,2020-05-11 10:50:52,http://www.ddaily.co.kr/news/article/?no=195334,...,None,None,None,NaN,2020-05-11 21:50:32,Y,Y,Y,2021-05-28,2021-06-02
3,7,삼성전자,LCD 외부조달 해도 TV 1위 문제없다.. 삼성의 '자신감',삼성전자(005930) 소비자가전(CE) 사업부의 디스플레이 구매 비용이 최근 2년...,이 때문에 내년부터 자회사 삼성디스플레이로부터 액정표시장치(LCD) 를 공급받지 못...,"< 저작권자 ⓒ 서울경제, 무단 전재 및 재배포 금지 >\n\n소비자가전(CE) 사...",서울경제,2020-05-11,2020-05-11 07:50:56,https://www.sedaily.com/NewsVIew/1Z2PIYWLFO,...,None,None,None,1.0,2020-05-11 21:50:32,Y,Y,Y,2021-05-28,2021-06-02
4,8,삼성전자,"삼성전자 스마트워치, 美 포드 자동차 공장 재가동 돕는다",앞서 삼성전자는 포드와 스타트업 육성 등 분야에서 파트너십 관계를 유지하고 있다. ...,앞서 삼성전자는 포드와 스타트업 육성 등 분야에서 파트너십 관계를 유지하고 있다. ...,앞서 삼성전자는 포드와 스타트업 육성 등 분야에서 파트너십 관계를 유지하고 있다. ...,아주경제_모바일,2020-05-11,2020-05-11 00:50:56,https://www.ajunews.com/view/20200510123114665,...,None,None,None,1.0,2020-05-11 21:50:32,Y,Y,Y,2021-05-28,2021-06-02
5,12,삼성전자,"삼성전자, '더 세로' TV·'비스포크' 냉장고 이달 중 해외 공략","더 세로는 미국, 비스포크는 중국…하반기 출시국 확대 예정. [아이뉴스24 윤선훈 ...","11일 삼성전자에 따르면 삼성전자는 '더 세로'를 이달 중 미국에, '비스포크'는 ...",[아이뉴스24 윤선훈 기자] 삼성전자가 이달 라이프스타일 TV '더 세로'와 맞춤형...,아이뉴스24,2020-05-11,2020-05-11 09:51:02,http://www.inews24.com/view/1263816,...,None,None,None,NaN,2020-05-11 21:50:32,Y,Y,Y,2021-05-28,2021-06-02
6,13,삼성전자,"애플은 더 싸게 내놓고, 교체수요는 뚝… 위기의 삼성폰",삼성전자 스마트폰 사업에 위기감이 높아지고 있다. 코로나 팬데믹(대유행)에 직격탄을...,"업계 관계자는 ""이는 삼성의 첫 갤럭시폰인 '갤럭시S'의 판매량(2500만대)에도 ...",삼성전자 스마트폰 사업에 위기감이 높아지고 있다. 코로나 팬데믹(대유행)에 직격탄을...,뉴스플러스,2020-05-11,2020-05-11 03:51:02,https://news.chosun.com/site/data/html_dir/202...,...,시장기회 및 재무성과,부정,,NaN,2020-05-11 21:50:32,Y,Y,Y,2021-05-28,2021-06-02
7,18,삼성전자,5주년 맞은 삼성페이의 진화... 비대면 결제시장서 '기회',삼성전자는 지난 2015년 상반기 플래그십폰 갤럭시S6 출시를 한달 여 앞둔 상황에...,삼성전자는 지난 2015년 상반기 플래그십폰 갤럭시S6 출시를 한달 여 앞둔 상황에...,삼성전자는 지난 2015년 상반기 플래그십폰 갤럭시S6 출시를 한달 여 앞둔 상황에...,뉴데일리경제,2020-05-11,2020-05-11 13:51:02,http://biz.newdaily.co.kr/site/data/html/2020/...,...,시장기회 및 재무성과,긍정,,NaN,2020-05-11 21:50:32,Y,Y,Y,2021-05-28,2021-06-02
8,19,삼성전자,"`경영권 불승계`…""삼성물산·전자, 기업가치 재평가 기대""",정 연구원은 “이재용 부회장은 삼성물산에 대한 최대주주(지분 17.2%) 지위를 확...,정 연구원은 “이재용 부회장은 삼성물산에 대한 최대주주(지분 17.2%) 지위를 확...,[이데일리 김재은 기자] 미래에셋대우는 11일 삼성물산(028260) 삼성전자(00...,이데일리 SPN,2020-05-11,2020-05-11 08:51:02,https://www.edaily.co.kr/news/read?newsId=0144...,...,주주가치 보호,긍정,,NaN,2020-05-11 21:50:32,Y,Y,Y,2021-05-28,2021-06-02
9,21,삼성전자,"목소리 높인 삼성에스원노조 ""임금인상하라""...삼성화재애니카 ...",[서울=뉴시스]전신 기자 = 11일 서울 서초구 삼성전자 서초사옥 앞에서 삼성에스원...,"울=뉴시스]전신 기자 = 11일 서울 서초구 삼성전자 서초사옥 앞에서 삼성에스원, ...","""올해 임협 결렬…임금 현실적 인상하라""\n\n에스원 ""열린 자세로 노조와 대화할 ...",뉴시스,2020-05-12,2020-05-11 17:51:10,https://newsis.com/view?id=NISX20200511_000102...,...,None,None,None,NaN,2020-05-11 21:50:32,Y,Y,Y,2021-05-28,2021-06-02


In [10]:
df_na['label_category'].apply(lambda x : x.replace(' ','') if x is not None else x).nunique()

46

In [11]:
df_na['label_category'].value_counts()

                 553086
시장기회 및 재무성과       24987
기타                21642
기후변화 및 에너지         8056
사회공헌               6810
노동관행/인적자원관리        5591
소유구조               4565
오픈이노베이션            4359
제품/서비스 책임          3867
지속가능성 거버넌스         2956
동반성장               2519
사업장 안전 및 보건        2353
내부통제/투명성/윤리경영      2107
소비자                2003
이사회 구성과 운영         1963
지역사회 영향            1816
고용                 1779
독과점/공정거래           1758
인수합병               1506
폐기물 관리             1292
계열회사 거래             946
감사기구                695
공공정책 참여             595
이익배분                504
인권                  473
주주가치 보호             408
자원 및 환경오염           332
대기오염                266
공시                  239
공급망 관리              237
주주가치보호              235
환경관리 체계             194
수질오염                112
화학물질                 92
친환경제품/서비스 개발         80
생태영향                 54
자원절약                 35
기타오염                 33
토양오염                 19
환경관리 체계               8
독과점/공정거래/윤리경영         5
환경경영(환경관리시스템)   

**na news는 QESG에서 예측한 Label**

### Read media_news Parquet

In [12]:
df_media_news = pq.read_pandas('df_media_news.parquet').to_pandas()

In [13]:
df_media_news.head(10)

,id,title,summary,media,date,link,category_predict,sentiment_predict,label_category,label_sentiment,label_category_etc
0,1,"美, 반도체 리쇼어링 속도… 파운드리 갈 길 바쁜 삼성전자에 기회?",코로나로 반도체 글로벌 공급망 위태… 자국 내 위탁생산시설 설립 추진 박차 1위 T...,뉴스플러스,2020-05-11,https://biz.chosun.com/site/data/html_dir/2020...,0.90,0.590000,제외,긍정,X
1,5,"[OA칼럼] 삼성전자, D램 EUV 공정 적용 의미와 전망은",편집자 주 : 전자부품 전문미디어 디일렉은 미국 메모리 반도체 전문 시장조사업체 오...,디일렉,2020-05-12,http://www.thelec.kr/news/articleView.html?idx...,0.51,0.610000,제외,부정,X
2,6,"“36개월간 안전하게” KT, 삼성전자 단말보험 '갤럭시케어' 출시",[디지털데일리 최민지기자] KT가 삼성전자 전용 단말보험을 내놓았다. 자기부담금은 ...,디지털데일리,2020-05-11,http://www.ddaily.co.kr/news/article/?no=195334,0.82,1.000000,제외,긍정,X
3,7,LCD 외부조달 해도 TV 1위 문제없다.. 삼성의 '자신감',삼성전자(005930) 소비자가전(CE) 사업부의 디스플레이 구매 비용이 최근 2년...,서울경제,2020-05-11,https://www.sedaily.com/NewsVIew/1Z2PIYWLFO,0.95,0.530000,제외,긍정,X
4,8,"삼성전자 스마트워치, 美 포드 자동차 공장 재가동 돕는다",앞서 삼성전자는 포드와 스타트업 육성 등 분야에서 파트너십 관계를 유지하고 있다. ...,아주경제_모바일,2020-05-11,https://www.ajunews.com/view/20200510123114665,0.81,0.630000,제외,긍정,X
5,12,"삼성전자, '더 세로' TV·'비스포크' 냉장고 이달 중 해외 공략","더 세로는 미국, 비스포크는 중국…하반기 출시국 확대 예정. [아이뉴스24 윤선훈 ...",아이뉴스24,2020-05-11,http://www.inews24.com/view/1263816,0.90,0.950000,제외,긍정,X
6,13,"애플은 더 싸게 내놓고, 교체수요는 뚝… 위기의 삼성폰",삼성전자 스마트폰 사업에 위기감이 높아지고 있다. 코로나 팬데믹(대유행)에 직격탄을...,뉴스플러스,2020-05-11,https://news.chosun.com/site/data/html_dir/202...,0.99,0.790000,제외,부정,X
7,18,5주년 맞은 삼성페이의 진화... 비대면 결제시장서 '기회',삼성전자는 지난 2015년 상반기 플래그십폰 갤럭시S6 출시를 한달 여 앞둔 상황에...,뉴데일리경제,2020-05-11,http://biz.newdaily.co.kr/site/data/html/2020/...,0.86,0.940299,제외,긍정,X
8,19,"`경영권 불승계`…삼성물산·전자, 기업가치 재평가 기대",정 연구원은 “이재용 부회장은 삼성물산에 대한 최대주주(지분 17.2%) 지위를 확...,이데일리 SPN,2020-05-11,https://www.edaily.co.kr/news/read?newsId=0144...,0.65,0.890000,주주가치 보호,긍정,X
9,21,목소리 높인 삼성에스원노조 임금인상하라...삼성화재애니카 ...,[서울=뉴시스]전신 기자 = 11일 서울 서초구 삼성전자 서초사옥 앞에서 삼성에스원...,뉴시스,2020-05-12,https://newsis.com/view?id=NISX20200511_000102...,1.00,1.000000,인적자원관리,부정,X


In [14]:
df_media_news['label_category'].value_counts()

제외              146627
사회공헌             20923
에너지 및 온실가스       19209
인적자원관리           15931
소유구조             13788
제품/서비스            9039
사업장 안전 및 보건       5887
ESG 거버넌스          5576
내부통제/투명성          4821
공급망 관리            3809
이사회 구성과 운영        3435
자원순환              3048
소비자 보호            2989
지역사회 영향           2858
공정거래              2372
이익배분              1083
계열회사와의 거래          920
감사기구               837
인권                 506
대기오염               453
주주가치 보호            372
환경관리 체계            207
공시                 196
수질오염               109
화학물질 배출             85
친환경제품/서비스 개발        75
생물다양성               61
토양오염                13
Name: label_category, dtype: int64

### Read qesg28_final Parquet

In [15]:
df_qesg28 = pq.read_pandas('df_qesg28_final.pq').to_pandas()

In [16]:
df_qesg28.head(5)

,id,keyword,title,link,contents,y,contents_clean
0,5,삼성전자,"[OA칼럼] 삼성전자, D램 EUV 공정 적용 의미와 전망은",http://www.thelec.kr/news/articleView.html?idx...,파운드리 사업과 장비 사용 공유\n\n편집자 주 : 전자부품 전문미디어 디일렉은 미...,제외,파운드리 사업과 장비 사용 공유편집자 주 전자부품 전문미디어 디일렉은 미국 메모리 ...
1,13,삼성전자,"애플은 더 싸게 내놓고, 교체수요는 뚝… 위기의 삼성폰",https://news.chosun.com/site/data/html_dir/202...,삼성전자 스마트폰 사업에 위기감이 높아지고 있다. 코로나 팬데믹(대유행)에 직격탄을...,제외,삼성전자 스마트폰 사업에 위기감이 높아지고 있다. 코로나 팬데믹에 직격탄을 맞아 판...
2,18,삼성전자,5주년 맞은 삼성페이의 진화... 비대면 결제시장서 '기회',http://biz.newdaily.co.kr/site/data/html/2020/...,삼성전자는 지난 2015년 상반기 플래그십폰 갤럭시S6 출시를 한달 여 앞둔 상황에...,제외,삼성전자는 지난 2015년 상반기 플래그십폰 갤럭시s6 출시를 한달 여 앞둔 상황에...
3,19,삼성전자,"`경영권 불승계`…""삼성물산·전자, 기업가치 재평가 기대""",https://www.edaily.co.kr/news/read?newsId=0144...,[이데일리 김재은 기자] 미래에셋대우는 11일 삼성물산(028260) 삼성전자(00...,주주가치보호,미래에셋대우는 11일 삼성물산 삼성전자 등 삼성그룹주의 기업가치 재평가가 기대된다고...
4,30,삼성전자,이건희 회장 병상 6년… 이재용표 '뉴 삼성' 도약할까,https://www.seoul.co.kr/news/newsView.php?id=2...,李 회장 여전히 의식 없지만 자가 호흡\n\n▲ 이재용 삼성전자 부회장.\n\n연합...,소유구조,회장 여전히 의식 없지만 자가 호흡 이재용 삼성전자 부회장.이건희 삼성전자 회장이 ...


In [17]:
df_qesg28_final = df_qesg28[(df_qesg28['y'] != '제외') & (df_qesg28['y'] != '기타') & (df_qesg28['y'] != '기타오염')]

In [18]:
df_qesg28_final = df_qesg28_final.reset_index(drop=True)

In [19]:
len(df_qesg28_final)

54219

In [20]:
df_qesg28_final['y'].nunique()

27

In [21]:
df_qesg28_final['y'].value_counts() 

에너지및온실가스       8056
인적자원관리         7205
사회공헌           6798
소유구조           5971
제품/서비스         3637
ESG거버넌스        2956
공급망관리          2681
사업장안전및보건       2275
내부통제/투명성       2012
소비자보호          1942
이사회구성과운영       1873
지역사회영향         1768
공정거래           1687
자원순환           1287
계열회사와의거래        876
감사기구            648
주주가치보호          616
이익배분            492
인권              433
대기오염            255
환경관리체계          209
공시              205
수질오염            105
화학물질배출           83
친환경제품/서비스개발      80
생물다양성            54
토양오염             15
Name: y, dtype: int64

## EDA

### Dates

df_na에 있는 뉴스 중 df_qesg28_final에 있는 것만 추출하기

In [114]:
df_na_only_esg = df_na[df_na['id'].isin(df_qesg28_final['id'])]

In [120]:
df_na_only_esg['date'].min()

Timestamp('2016-06-01 00:00:00')

In [116]:
df_na_only_esg['date'].max()

Timestamp('2021-04-28 00:00:00')

In [118]:
df_na_only_esg_year = df_na_only_esg ['date'].apply(lambda x: x.year).to_frame()

In [127]:
df_na_only_esg_year.value_counts()

date
2020    25708
2021    19172
2018     4717
2019     3332
2017      871
2016      419
dtype: int64

### Companies

In [91]:
df_qesg28['keyword'].unique().tolist()

['삼성전자',
 'CJ CGV',
 'LF',
 'AK홀딩스',
 'CJ',
 'CJ대한통운',
 'CJ제일제당',
 'DB손해보험',
 'GS',
 'GS건설',
 'GS리테일',
 'HDC',
 'HDC현대산업개발',
 'JW중외제약',
 'JW홀딩스',
 'KB금융',
 'KCC',
 'KT',
 'KT&G',
 'LG',
 'LG디스플레이',
 'LG상사',
 'LG화학',
 'LS',
 'NAVER',
 'NH투자증권',
 'OCI',
 'S&T모티브',
 'SK',
 'SKC',
 'SK네트웍스',
 'SK이노베이션',
 'SK케미칼',
 'SK텔레콤',
 'S-Oil',
 '강원랜드',
 '기아차',
 '대상',
 '기업은행',
 '넷마블',
 '대웅제약',
 '대한유화',
 '대한항공',
 '동양',
 '두산중공업',
 '롯데지주',
 '롯데하이마트',
 '메리츠종금증권',
 '삼성SDI',
 '삼성물산',
 '삼성바이오로직스',
 '삼성생명',
 '삼성전기',
 '삼성중공업',
 '삼성증권',
 '삼성카드',
 '삼성화재',
 '신세계',
 '쌍용차',
 '우리금융지주',
 '팬오션',
 '하나금융지주',
 '한국가스공사',
 '한국전력',
 '한국콜마',
 '현대백화점',
 '한국항공우주',
 '한전기술',
 '한화',
 '한화에어로스페이스',
 '현대건설',
 '현대제철',
 '현대차',
 'BNK금융지주',
 'LG유플러스',
 'POSCO',
 '녹십자',
 '대우건설',
 '대우조선해양',
 '롯데케미칼',
 '동서',
 '롯데쇼핑',
 '미래에셋대우',
 '부광약품',
 '신한지주',
 '이마트',
 '카카오',
 '한국금융지주',
 '한국타이어앤테크놀로지',
 '한국테크놀로지그룹',
 '한화생명',
 '현대글로비스',
 '현대홈쇼핑',
 '효성',
 'LG전자',
 'SK하이닉스',
 '넥센타이어',
 '롯데칠성',
 '롯데푸드',
 '삼성에스디에스',
 '신세계인터내셔날',
 '아모레퍼시픽',
 '엔씨소프트',
 '오리온',
 

In [89]:
df_qesg28['keyword'].nunique()

1020

In [128]:
df_qesg28['keyword'].value_counts()

삼성전자      2696
현대자동차     1629
기업은행      1506
한국가스공사    1456
한국전력      1363
          ... 
삼현철강         1
삼화왕관         1
삼화전기         1
삼화페인트        1
한국 GM        1
Name: keyword, Length: 1020, dtype: int64

### Extracting Nouns

In [22]:
df_qesg28_final['contents_nouns'] = df_qesg28_final['contents_clean'].apply(lambda x:mecab.nouns(x))

In [23]:
df_qesg28_final['nouns_counter'] = df_qesg28_final['contents_nouns'].apply(lambda x: Counter(x))

In [24]:
df_qesg28_final['nouns_no']= df_qesg28_final['contents_nouns'].apply(lambda x: len(x))

In [25]:
df_qesg28_final.head(5) 

,id,keyword,title,link,contents,y,contents_clean,contents_nouns,nouns_counter,nouns_no
0,19,삼성전자,"`경영권 불승계`…""삼성물산·전자, 기업가치 재평가 기대""",https://www.edaily.co.kr/news/read?newsId=0144...,[이데일리 김재은 기자] 미래에셋대우는 11일 삼성물산(028260) 삼성전자(00...,주주가치보호,미래에셋대우는 11일 삼성물산 삼성전자 등 삼성그룹주의 기업가치 재평가가 기대된다고...,"[미래, 셋, 대우, 일, 삼성물산, 삼성전자, 등, 삼성그룹, 주, 기업, 가치,...","{'미래': 2, '셋': 2, '대우': 2, '일': 1, '삼성물산': 7, ...",216
1,30,삼성전자,이건희 회장 병상 6년… 이재용표 '뉴 삼성' 도약할까,https://www.seoul.co.kr/news/newsView.php?id=2...,李 회장 여전히 의식 없지만 자가 호흡\n\n▲ 이재용 삼성전자 부회장.\n\n연합...,소유구조,회장 여전히 의식 없지만 자가 호흡 이재용 삼성전자 부회장.이건희 삼성전자 회장이 ...,"[회장, 의식, 자, 호흡, 이재용, 삼성전자, 부회장, 이건희, 삼성전자, 회장,...","{'회장': 6, '의식': 2, '자': 2, '호흡': 2, '이재용': 2, ...",232
2,31,삼성전자,이태원발 코로나에 기업들 '긴장'…직원들에 자진검사 권고,https://www.yna.co.kr/view/AKR20200511125300003,"삼성, 온라인 문진에 '이태원 방문' 항목 추가 LG·SK·현대차 등 ""검사 후 보...",사업장안전및보건,삼성 온라인 문진에 이태원 방문 항목 추가 lgsk현대차 등 검사 후 보고하라낙인 ...,"[삼성, 온라인, 문진, 이태원, 방문, 항목, 추가, 현대차, 등, 검사, 후, ...","{'삼성': 1, '온라인': 1, '문진': 1, '이태원': 5, '방문': 3...",123
3,53,삼성전자,"검찰, 유상호 한국투자증권 부회장 '삼성 합병' 의혹 참고인 소환",http://www.ilemonde.com/news/articleView.html?...,"유상호 한국투자증권 대표. /사진=뉴스1 삼성물산과 제일모직 합병, 삼성바이오로직스...",내부통제/투명성,유상호 한국투자증권 대표. 사진뉴스1 삼성물산과 제일모직 합병 삼성바이오로직스 분식...,"[유상호, 한국, 투자, 증권, 대표, 사진, 뉴스, 삼성물산, 제일모직, 합병, ...","{'유상호': 3, '한국': 4, '투자': 4, '증권': 4, '대표': 3,...",233
4,79,삼성전자,"[who] 삼성·현대차·LG화학, 미래 비전 위한 혁신선언",https://cnbc.sbs.co.kr/article/10000982395,"오수영 기자, 먼저 이재용 삼성전자 부회장이 '4대 세습경영' 포기를 공식 선언했죠...",소유구조,먼저 이재용 삼성전자 부회장이 4대 세습경영 포기를 공식 선언했죠 ... 날 인도 ...,"[이재용, 삼성전자, 부회장, 대, 세습, 경영, 포기, 공식, 선언, 날, 인도,...","{'이재용': 1, '삼성전자': 1, '부회장': 1, '대': 1, '세습': ...",20


### Groupby w.r.t Categories

In [26]:
df_category = df_qesg28_final.groupby('y')['contents_nouns'].apply(list).to_frame()

In [27]:
df_category 

,contents_nouns
y,
ESG거버넌스,"[[남양유업, 대리점, 갑, 논란, 경쟁사, 비방, 댓글, 이슈, 주가, 하락, 기..."
감사기구,"[[서울, 북부, 지검, 일, 오전, 시, 분, 국세청, 고발, 관련, 그룹, 계열..."
계열회사와의거래,"[[지난해, 국내, 기업, 집단, 계열사, 간, 내부, 거래, 금액, 조, 원, 규..."
공급망관리,"[[롯데케미칼, 발간, 동반성, 롯데케미칼, 창사, 이래, 처음, 동반, 성장, 보..."
공시,"[[해외, 계열사, 지분, 허위, 공시, 혐의, 벌금형, 선고, 롯데, 그룹, 계열..."
공정거래,"[[중견, 기업, 한국, 테크놀로지, 한국, 타이어, 월드, 이드, 한국, 테크놀로..."
내부통제/투명성,"[[유상호, 한국, 투자, 증권, 대표, 사진, 뉴스, 삼성물산, 제일모직, 합병,..."
대기오염,"[[상황, 자동차, 회사, 국내, 수입, 미래, 차, 개발, 박차, 국내, 자동차,..."
사업장안전및보건,"[[삼성, 온라인, 문진, 이태원, 방문, 항목, 추가, 현대차, 등, 검사, 후,..."


In [28]:
df_category['number_of_news'] = df_category['contents_nouns'].apply(lambda x:len(x))

In [29]:
df_category['contents_nouns'] = df_category['contents_nouns'].apply(lambda x: flattened_list(x))

In [30]:
df_category['nouns_number_per_category'] = df_category['contents_nouns'].apply(lambda x:len(x))

In [31]:
df_category_wl = copy.deepcopy(df_category)

In [32]:
df_category_wl['contents_nouns'] = df_category_wl['contents_nouns'].apply(lambda x:' '.join(x))

In [33]:
df_category_wl

,contents_nouns,number_of_news,nouns_number_per_category
y,,,
ESG거버넌스,남양유업 대리점 갑 논란 경쟁사 비방 댓글 이슈 주가 하락 기업 투자 부각 환경 사...,2956,839444
감사기구,서울 북부 지검 일 오전 시 분 국세청 고발 관련 그룹 계열사 동제 련 압수수색 서...,648,163278
계열회사와의거래,지난해 국내 기업 집단 계열사 간 내부 거래 금액 조 원 규모 전체 매출 가량 차지...,876,245082
공급망관리,롯데케미칼 발간 동반성 롯데케미칼 창사 이래 처음 동반 성장 보고서 롯데케미칼 일 ...,2681,608235
공시,해외 계열사 지분 허위 공시 혐의 벌금형 선고 롯데 그룹 계열사 항소심 재판 혐의 ...,205,38779
공정거래,중견 기업 한국 테크놀로지 한국 타이어 월드 이드 한국 테크놀로지 한국 타이어 지주...,1687,422858
내부통제/투명성,유상호 한국 투자 증권 대표 사진 뉴스 삼성물산 제일모직 합병 삼성 바이오 로직스 ...,2012,473754
대기오염,상황 자동차 회사 국내 수입 미래 차 개발 박차 국내 자동차 시장 위 현대 자동차 ...,255,55505
사업장안전및보건,삼성 온라인 문진 이태원 방문 항목 추가 현대차 등 검사 후 보고 낙인 효과 우려 ...,2275,468883


#### Groupby w.r.t Categoreis w.o Duplicates

In [92]:
df_category_wo_dup = df_category['contents_nouns'].apply(lambda x:list(dict.fromkeys(x))).to_frame()
df_category_wo_dup['nouns_number_per_category'] = df_category_wo_dup['contents_nouns'].apply(lambda x:len(x)) 
df_category_wo_dup_wl = copy.deepcopy(df_category_wo_dup)
df_category_wo_dup_wl['contents_nouns'] = df_category_wo_dup_wl['contents_nouns'].apply(lambda x:' '.join(x))
df_category_wo_dup_wl

,contents_nouns,nouns_number_per_category
y,,
ESG거버넌스,남양유업 대리점 갑 논란 경쟁사 비방 댓글 이슈 주가 하락 기업 투자 부각 환경 사...,16638
감사기구,서울 북부 지검 일 오전 시 분 국세청 고발 관련 그룹 계열사 동제 련 압수수색 지...,8284
계열회사와의거래,지난해 국내 기업 집단 계열사 간 내부 거래 금액 조 원 규모 전체 매출 가량 차지...,9252
공급망관리,롯데케미칼 발간 동반성 창사 이래 처음 동반 성장 보고서 일 장 전략 년 관련 성과...,16085
공시,해외 계열사 지분 허위 공시 혐의 벌금형 선고 롯데 그룹 항소심 재판 전면 부인 서...,3790
공정거래,중견 기업 한국 테크놀로지 타이어 월드 이드 지주사 그룹 대상 상호 사용 금지 등 ...,13349
내부통제/투명성,유상호 한국 투자 증권 대표 사진 뉴스 삼성물산 제일모직 합병 삼성 바이오 로직스 ...,16074
대기오염,상황 자동차 회사 국내 수입 미래 차 개발 박차 시장 위 현대 내년 순수 전기 본격...,4578
사업장안전및보건,삼성 온라인 문진 이태원 방문 항목 추가 현대차 등 검사 후 보고 낙인 효과 우려 ...,15400


### Groupby w.r.t Categories (multiplicity greater than one)

In [78]:
df_category_m_g_one = df_category['contents_nouns'].apply(lambda x: remove_fewer_multiplicity_elements(x, 1)).to_frame()
df_category_m_g_one['nouns_number_per_category'] = df_category_m_g_one['contents_nouns'].apply(lambda x:len(x)) 
df_category_m_g_one_wl = copy.deepcopy(df_category_m_g_one)
df_category_m_g_one_wl['contents_nouns'] = df_category_m_g_one_wl['contents_nouns'].apply(lambda x:' '.join(x))
df_category_m_g_one_wl

,contents_nouns,nouns_number_per_category
y,,
ESG거버넌스,남양유업 대리점 갑 논란 경쟁사 댓글 이슈 주가 하락 기업 투자 부각 환경 사회 기...,834356
감사기구,서울 북부 지검 일 오전 시 분 국세청 고발 관련 그룹 계열사 동제 련 압수수색 서...,160396
계열회사와의거래,지난해 국내 기업 집단 계열사 간 내부 거래 금액 조 원 규모 전체 매출 가량 차지...,241987
공급망관리,롯데케미칼 발간 동반성 롯데케미칼 창사 이래 처음 동반 성장 보고서 롯데케미칼 일 ...,603046
공시,해외 계열사 지분 허위 공시 혐의 벌금형 선고 롯데 그룹 계열사 항소심 재판 혐의 ...,37457
공정거래,중견 기업 한국 테크놀로지 한국 타이어 월드 이드 한국 테크놀로지 한국 타이어 지주...,418630
내부통제/투명성,유상호 한국 투자 증권 대표 사진 뉴스 삼성물산 제일모직 합병 삼성 바이오 로직스 ...,468404
대기오염,상황 자동차 회사 국내 수입 미래 차 개발 박차 국내 자동차 시장 위 현대 자동차 ...,53879
사업장안전및보건,삼성 온라인 문진 이태원 방문 항목 추가 현대차 등 검사 후 보고 낙인 효과 우려 ...,463659


#### Groupby w.r.t Categories (multiplicity greater than one) w.o Duplicates

In [80]:
df_category_m_g_one_wo_dup = df_category_m_g_one['contents_nouns'].apply(lambda x:list(dict.fromkeys(x))).to_frame()
df_category_m_g_one_wo_dup['nouns_number_per_category'] = df_category_m_g_one_wo_dup['contents_nouns'].apply(lambda x:len(x)) 
df_category_m_g_one_wo_dup_wl = copy.deepcopy(df_category_m_g_one_wo_dup)  
df_category_m_g_one_wo_dup_wl['contents_nouns'] = df_category_m_g_one_wo_dup_wl['contents_nouns'].apply(lambda x:' '.join(x))
df_category_m_g_one_wo_dup_wl

,contents_nouns,nouns_number_per_category
y,,
ESG거버넌스,남양유업 대리점 갑 논란 경쟁사 댓글 이슈 주가 하락 기업 투자 부각 환경 사회 기...,11550
감사기구,서울 북부 지검 일 오전 시 분 국세청 고발 관련 그룹 계열사 동제 련 압수수색 지...,5402
계열회사와의거래,지난해 국내 기업 집단 계열사 간 내부 거래 금액 조 원 규모 전체 매출 가량 차지...,6157
공급망관리,롯데케미칼 발간 동반성 창사 이래 처음 동반 성장 보고서 일 장 전략 년 관련 성과...,10896
공시,해외 계열사 지분 허위 공시 혐의 벌금형 선고 롯데 그룹 항소심 재판 서울 중앙 지...,2468
공정거래,중견 기업 한국 테크놀로지 타이어 월드 이드 지주사 그룹 대상 상호 사용 금지 등 ...,9121
내부통제/투명성,유상호 한국 투자 증권 대표 사진 뉴스 삼성물산 제일모직 합병 삼성 바이오 로직스 ...,10724
대기오염,상황 자동차 회사 국내 수입 미래 차 개발 박차 시장 위 현대 내년 순수 전기 본격...,2952
사업장안전및보건,삼성 온라인 문진 이태원 방문 항목 추가 현대차 등 검사 후 보고 낙인 효과 우려 ...,10176


### Groupby w.r.t Categories (multiplicity greater than two)

In [79]:
df_category_m_g_two = df_category['contents_nouns'].apply(lambda x: remove_fewer_multiplicity_elements(x, 2)).to_frame()
df_category_m_g_two['nouns_number_per_category'] = df_category_m_g_two['contents_nouns'].apply(lambda x:len(x)) 
df_category_m_g_two_wl = copy.deepcopy(df_category_m_g_two)
df_category_m_g_two_wl['contents_nouns'] = df_category_m_g_two_wl['contents_nouns'].apply(lambda x:' '.join(x))
df_category_m_g_two_wl

,contents_nouns,nouns_number_per_category
y,,
ESG거버넌스,남양유업 대리점 갑 논란 경쟁사 댓글 이슈 주가 하락 기업 투자 부각 환경 사회 기...,829754
감사기구,서울 북부 지검 일 오전 시 분 국세청 고발 관련 그룹 계열사 동제 련 압수수색 서...,158058
계열회사와의거래,지난해 국내 기업 집단 계열사 간 내부 거래 금액 조 원 규모 전체 매출 가량 차지...,239397
공급망관리,롯데케미칼 발간 동반성 롯데케미칼 창사 이래 처음 동반 성장 보고서 롯데케미칼 일 ...,598710
공시,해외 계열사 지분 허위 공시 혐의 벌금형 선고 롯데 그룹 계열사 항소심 재판 혐의 ...,36161
공정거래,중견 기업 한국 테크놀로지 한국 타이어 월드 이드 한국 테크놀로지 한국 타이어 지주...,414852
내부통제/투명성,유상호 한국 투자 증권 대표 사진 뉴스 삼성물산 제일모직 합병 삼성 바이오 로직스 ...,464010
대기오염,상황 자동차 회사 국내 수입 미래 차 개발 박차 국내 자동차 시장 위 현대 자동차 ...,52401
사업장안전및보건,삼성 온라인 문진 이태원 방문 항목 추가 현대차 등 검사 후 보고 낙인 효과 우려 ...,459463


#### Groupby w.r.t Categories (multiplicity greater than two) w.o Duplicates

In [81]:
df_category_m_g_two_wo_dup = df_category_m_g_two['contents_nouns'].apply(lambda x:list(dict.fromkeys(x))).to_frame()
df_category_m_g_two_wo_dup['nouns_number_per_category'] = df_category_m_g_two_wo_dup['contents_nouns'].apply(lambda x:len(x)) 
df_category_m_g_two_wo_dup_wl = copy.deepcopy(df_category_m_g_two_wo_dup)  
df_category_m_g_two_wo_dup_wl['contents_nouns'] = df_category_m_g_two_wo_dup_wl['contents_nouns'].apply(lambda x:' '.join(x))
df_category_m_g_two_wo_dup_wl

,contents_nouns,nouns_number_per_category
y,,
ESG거버넌스,남양유업 대리점 갑 논란 경쟁사 댓글 이슈 주가 하락 기업 투자 부각 환경 사회 기...,9249
감사기구,서울 북부 지검 일 오전 시 분 국세청 고발 관련 그룹 계열사 동제 련 압수수색 지...,4233
계열회사와의거래,지난해 국내 기업 집단 계열사 간 내부 거래 금액 조 원 규모 전체 매출 가량 차지...,4862
공급망관리,롯데케미칼 발간 동반성 창사 이래 처음 동반 성장 보고서 일 장 전략 년 관련 성과...,8728
공시,해외 계열사 지분 허위 공시 혐의 벌금형 선고 롯데 그룹 항소심 재판 서울 중앙 형...,1820
공정거래,중견 기업 한국 테크놀로지 타이어 월드 이드 지주사 그룹 대상 상호 사용 금지 등 ...,7232
내부통제/투명성,유상호 한국 투자 증권 대표 사진 뉴스 삼성물산 제일모직 합병 삼성 바이오 로직스 ...,8527
대기오염,상황 자동차 회사 국내 수입 미래 차 개발 박차 시장 위 현대 내년 순수 전기 본격...,2213
사업장안전및보건,삼성 온라인 문진 이태원 방문 항목 추가 현대차 등 검사 후 보고 낙인 효과 우려 ...,8078


### Groupby w.r.t Categories (multiplicity greater than three)

In [82]:
df_category_m_g_three = df_category['contents_nouns'].apply(lambda x: remove_fewer_multiplicity_elements(x, 3)).to_frame()
df_category_m_g_three['nouns_number_per_category'] = df_category_m_g_three['contents_nouns'].apply(lambda x:len(x)) 
df_category_m_g_three_wl = copy.deepcopy(df_category_m_g_three)
df_category_m_g_three_wl['contents_nouns'] = df_category_m_g_three_wl['contents_nouns'].apply(lambda x:' '.join(x))
df_category_m_g_three_wl

,contents_nouns,nouns_number_per_category
y,,
ESG거버넌스,남양유업 대리점 갑 논란 경쟁사 댓글 이슈 주가 하락 기업 투자 부각 환경 사회 기...,826094
감사기구,서울 북부 지검 일 오전 시 분 국세청 고발 관련 그룹 계열사 동제 련 압수수색 서...,156036
계열회사와의거래,지난해 국내 기업 집단 계열사 간 내부 거래 금액 조 원 규모 전체 매출 가량 차지...,237294
공급망관리,롯데케미칼 발간 동반성 롯데케미칼 창사 이래 처음 동반 성장 보고서 롯데케미칼 일 ...,595155
공시,해외 계열사 지분 허위 공시 혐의 선고 롯데 그룹 계열사 항소심 재판 혐의 서울 중...,35159
공정거래,중견 기업 한국 테크놀로지 한국 타이어 월드 이드 한국 테크놀로지 한국 타이어 지주...,411717
내부통제/투명성,유상호 한국 투자 증권 대표 사진 뉴스 삼성물산 제일모직 합병 삼성 바이오 로직스 ...,460179
대기오염,상황 자동차 회사 국내 수입 미래 차 개발 박차 국내 자동차 시장 위 현대 자동차 ...,51342
사업장안전및보건,삼성 온라인 문진 이태원 방문 항목 추가 현대차 등 검사 후 보고 낙인 효과 우려 ...,455887


#### Groupby w.r.t Categories (multiplicity greater than three) w.o Duplicates

In [83]:
df_category_m_g_three_wo_dup = df_category_m_g_two['contents_nouns'].apply(lambda x:list(dict.fromkeys(x))).to_frame()
df_category_m_g_three_wo_dup['nouns_number_per_category'] = df_category_m_g_three_wo_dup['contents_nouns'].apply(lambda x:len(x)) 
df_category_m_g_three_wo_dup_wl = copy.deepcopy(df_category_m_g_three_wo_dup)  
df_category_m_g_three_wo_dup_wl['contents_nouns'] = df_category_m_g_three_wo_dup_wl['contents_nouns'].apply(lambda x:' '.join(x))
df_category_m_g_three_wo_dup_wl

,contents_nouns,nouns_number_per_category
y,,
ESG거버넌스,남양유업 대리점 갑 논란 경쟁사 댓글 이슈 주가 하락 기업 투자 부각 환경 사회 기...,9249
감사기구,서울 북부 지검 일 오전 시 분 국세청 고발 관련 그룹 계열사 동제 련 압수수색 지...,4233
계열회사와의거래,지난해 국내 기업 집단 계열사 간 내부 거래 금액 조 원 규모 전체 매출 가량 차지...,4862
공급망관리,롯데케미칼 발간 동반성 창사 이래 처음 동반 성장 보고서 일 장 전략 년 관련 성과...,8728
공시,해외 계열사 지분 허위 공시 혐의 벌금형 선고 롯데 그룹 항소심 재판 서울 중앙 형...,1820
공정거래,중견 기업 한국 테크놀로지 타이어 월드 이드 지주사 그룹 대상 상호 사용 금지 등 ...,7232
내부통제/투명성,유상호 한국 투자 증권 대표 사진 뉴스 삼성물산 제일모직 합병 삼성 바이오 로직스 ...,8527
대기오염,상황 자동차 회사 국내 수입 미래 차 개발 박차 시장 위 현대 내년 순수 전기 본격...,2213
사업장안전및보건,삼성 온라인 문진 이태원 방문 항목 추가 현대차 등 검사 후 보고 낙인 효과 우려 ...,8078


In [85]:
df_category_m_g_three_wo_dup_wl['nouns_number_per_category'] == df_category_m_g_two_wo_dup_wl['nouns_number_per_category']

y
ESG거버넌스        True
감사기구           True
계열회사와의거래       True
공급망관리          True
공시             True
공정거래           True
내부통제/투명성       True
대기오염           True
사업장안전및보건       True
사회공헌           True
생물다양성          True
소비자보호          True
소유구조           True
수질오염           True
에너지및온실가스       True
이사회구성과운영       True
이익배분           True
인권             True
인적자원관리         True
자원순환           True
제품/서비스         True
주주가치보호         True
지역사회영향         True
친환경제품/서비스개발    True
토양오염           True
화학물질배출         True
환경관리체계         True
Name: nouns_number_per_category, dtype: bool

### Groupby w.r.t Categories (multiplicity greater than five)

In [86]:
df_category_m_g_five = df_category['contents_nouns'].apply(lambda x: remove_fewer_multiplicity_elements(x, 5)).to_frame()
df_category_m_g_five['nouns_number_per_category'] = df_category_m_g_five['contents_nouns'].apply(lambda x:len(x)) 
df_category_m_g_five_wl = copy.deepcopy(df_category_m_g_five)
df_category_m_g_five_wl['contents_nouns'] = df_category_m_g_five_wl['contents_nouns'].apply(lambda x:' '.join(x))
df_category_m_g_five_wl

,contents_nouns,nouns_number_per_category
y,,
ESG거버넌스,남양유업 대리점 갑 논란 경쟁사 댓글 이슈 주가 하락 기업 투자 부각 환경 사회 기...,819067
감사기구,서울 지검 일 오전 시 분 국세청 고발 관련 그룹 계열사 동제 련 압수수색 서울 지...,152747
계열회사와의거래,지난해 국내 기업 집단 계열사 간 내부 거래 금액 조 원 규모 전체 매출 가량 차지...,233613
공급망관리,롯데케미칼 발간 동반성 롯데케미칼 이래 처음 동반 성장 보고서 롯데케미칼 일 동반성...,588884
공시,해외 계열사 지분 허위 공시 혐의 선고 롯데 그룹 계열사 재판 혐의 서울 중앙 형사...,33510
공정거래,중견 기업 한국 테크놀로지 한국 타이어 월드 한국 테크놀로지 한국 타이어 지주사 한...,406414
내부통제/투명성,유상호 한국 투자 증권 대표 사진 뉴스 삼성물산 제일모직 합병 삼성 바이오 로직스 ...,453949
대기오염,상황 자동차 회사 국내 미래 차 개발 국내 자동차 시장 위 현대 자동차 내년 전기 ...,49348
사업장안전및보건,삼성 온라인 문진 이태원 방문 항목 추가 현대차 등 검사 후 보고 효과 우려 업계 ...,449522


#### Groupby w.r.t Categories (multiplicity greater than five) w.o Duplicates

In [87]:
df_category_m_g_five_wo_dup = df_category_m_g_five['contents_nouns'].apply(lambda x:list(dict.fromkeys(x))).to_frame()
df_category_m_g_five_wo_dup['nouns_number_per_category'] = df_category_m_g_five_wo_dup['contents_nouns'].apply(lambda x:len(x)) 
df_category_m_g_five_wo_dup_wl = copy.deepcopy(df_category_m_g_five_wo_dup)  
df_category_m_g_five_wo_dup_wl['contents_nouns'] = df_category_m_g_five_wo_dup_wl['contents_nouns'].apply(lambda x:' '.join(x))
df_category_m_g_five_wo_dup_wl

,contents_nouns,nouns_number_per_category
y,,
ESG거버넌스,남양유업 대리점 갑 논란 경쟁사 댓글 이슈 주가 하락 기업 투자 부각 환경 사회 기...,6439
감사기구,서울 지검 일 오전 시 분 국세청 고발 관련 그룹 계열사 동제 련 압수수색 지방 조...,2815
계열회사와의거래,지난해 국내 기업 집단 계열사 간 내부 거래 금액 조 원 규모 전체 매출 가량 차지...,3324
공급망관리,롯데케미칼 발간 동반성 이래 처음 동반 성장 보고서 일 장 전략 년 관련 성과 파트...,6122
공시,해외 계열사 지분 허위 공시 혐의 선고 롯데 그룹 재판 서울 중앙 형사 부 일 공정...,1112
공정거래,중견 기업 한국 테크놀로지 타이어 월드 지주사 그룹 대상 상호 사용 금지 등 가처분...,4987
내부통제/투명성,유상호 한국 투자 증권 대표 사진 뉴스 삼성물산 제일모직 합병 삼성 바이오 로직스 ...,5833
대기오염,상황 자동차 회사 국내 미래 차 개발 시장 위 현대 내년 전기 본격 생산 계획 양산...,1410
사업장안전및보건,삼성 온라인 문진 이태원 방문 항목 추가 현대차 등 검사 후 보고 효과 우려 업계 ...,5441


## Save Files

원하는 데이터 프레임을 인풋으로 넣으면 카테고리 별로 워드 파일이 DataFrame의 이름으로 된 folder에 저장 됨.